In [1]:
import mne
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import itertools

from IPython.utils import io

from ast import literal_eval

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### Common Spatial Patterns (CSP)

Next, we will implement and test CSP against our data to try and improve our predictive ability. In general, CSP is a signal processing technique (particularly for classification problems) in which multivariate signals (e.g., an EEG device with 30 electrodes, like we are using here) are separated into subcomponents which maximize the differences between the classes of signal.

In practice, this will collapse our dataset for each test from an array of 30 channels x thousands of samples to just a few vector values. We will perform a gridsearch to find the ideal number of vectors for our problem, as well as whether a neural network or Linear Discriminant Analysis is the best modeling tool to make class predictions based on those CSP vectors.

### First, let's ingest our data using the data ingester we built
After running, this script will load several dictionaries into memory, as well as other needed objects:
1. data_dict - All the data EEG recordings
2. event_dict - which indicates the sample number at which each stimulus was applied
3. y_dict - which has the type of experiment conducted in each trial
4. info -  file used to create MNE raw objects including channel names, type, and sampling frequency
5. events_explained - dictionary which provides the names for each of the five trial types
6. ch_names - list of all channel names

In [2]:
%run data_ingester.py

### Let's find the best parameters to use to create the clearest differentiation using CSP

In this grid search we'll be doing changing three major kinds of parameters to optimize our CSP settings: 

1. A smaller subset of the preprocessing options we tested using a CNN
2. Which trials to drop for each subject (the flatness and spikiness settings to use in rejecting each trial)
3. The CSP parameters to use in CSP feature extraction

Later, we will refine and iterate on the models we use to make predictions using this CSP data, but for now we will use a straightforward LDA to test which sets of parameters lead to the most differentiable CSP features.

Here is a full explanation of the CSP parameters and models we will be testing. For a full explanation of the preprocessing options, see the preprocessing options grid search notebook:

1. Preprocessing options
2. CSP parameters
    - Number of CSP components to create (n_components) and transform data into
    - Whether covariance matrices are created based on epochs concatenated together or on individual epochs and concatenated (cov_est)
    - Whether a log transform is applied to standardize features
4. Which pairwise combination of trials to compare
    - (1, 5) Word association vs imagining foot movement
    - (1, 4) Word association vs imagining hand movement
    - (2, 4) Mental subtraction vs imagining hand movement
    - (1, 3) Word association vs mental navigation
    
**Also note that these CSP models will be created individually (i.e., looking at the data for only one participant in training and validation)**

The grid search will save down the accuracy of an LDA and NN model on our test data from day 1 for each combination, and then we can select the highest-performing parameters to be used as the basis of the personalized L1 model for each subject.

In [3]:
#Preprocessing parameters to gridsearch
l_freq_filter_options = [None]
h_freq_filter_options = [40]
channels_to_drop_options = [['AFz', 'F7', 'F8']]
baseline_correction_options = [None]
projectors_to_apply_options = [slice(1)] #check that this generated best results
selected_frequency_options = [256]
tmin_options = [1] #Later start to avoid initialization of thought pattern
tmax_options = [4.5]
detrend_options = [None]
reject_options = [{'eeg': 150}] #Customize per subject later
flat_options = [{'eeg': 20}] #Customize per subject later
ica_to_exclude_options = [None] #Incorporate later if helpful in other gridsearch
scaler_options = ['robust', None] #Test no scaler for CSP
#CSP parameters to gridsearch
n_components_options = [4, 6, 8]
cov_est_options = ['concat', 'epoch']
log_options = [True, False]
#Combinations of trial types to compare
trial_combo_options = [(1, 5), (1, 4), (2, 4), (1, 3)]

In [4]:
#Create column names for test dataframe
columns = ['l_freq_filter',
           'h_freq_filter',
           'channels_to_drop',
           'baseline_correction',
           'projectors_to_apply',
           'selected_frequency',
           'tmin',
           'tmax',
           'detrend',
           'reject',
           'flat',
           'ica_to_exclude',
           'scaler', 
           'n_components',
           'cov_est', 
           'log',
           'trial_combo']

In [5]:
#Create dataframe with all combinations of tests as rows
test_df = pd.DataFrame(itertools.product(l_freq_filter_options, 
                                         h_freq_filter_options, 
                                         channels_to_drop_options, 
                                         baseline_correction_options, 
                                         projectors_to_apply_options, 
                                         selected_frequency_options,
                                         tmin_options,
                                         tmax_options, 
                                         detrend_options,
                                         reject_options,
                                         flat_options,
                                         ica_to_exclude_options,
                                         scaler_options, 
                                         n_components_options, 
                                         cov_est_options, 
                                         log_options, 
                                         trial_combo_options), 
                      columns=columns)

In [6]:
#Append columns for each subject, where we will record results for each test
subject_columns = ['sub_A', 'sub_C', 'sub_D', 'sub_E', 'sub_F', 'sub_G', 
                   'sub_H', 'sub_J', 'sub_L']

In [7]:
#Add those combos to our test_df to save highest val accuracy achieved
test_df = test_df.reindex(columns=columns + subject_columns)

In [8]:
test_df.shape

(96, 26)

### Let's test these options


First we will create a function to do our CSP grid searches, and then apply it to this first test dataframe.

In [15]:
def csp_grid_search(test_df, 
                    ind_or_group, 
                    savefile):
    """A function to iterate across a test
    dataframe and fill in the cross validated
    score in the results column.
    
    ind_or_group: 'ind', 'group'
    ind tests have one subject per row, 
    while group tests have each subject as 
    columns where accuracy is to be filled in.
    Individual level tests must have a row 
    called 'subject' which includes the subject
    to be tested on that row, e.g., sub_A.
    
    Savefile is the filename to use in saving
    test dataframe to csv in data directory
    each iteration."""
    for row in range(test_df.shape[0]):
        #Load each sessions data into an MNE raw object
        raw_dict = {}
        for key, value in data_dict.items():
            if ind_or_group == 'ind':
                if test_df.subject[row] in key:
                    raw_dict[key] = mne.io.RawArray(value.T, info, verbose=0)
            if ind_or_group == 'group':
                if 'sesh_1' in key:
                    raw_dict[key] = mne.io.RawArray(value.T, info, verbose=0)

        #Filter data with bandpass. Note raw.filter applies in place
        for key, value in raw_dict.items():
            value.filter(l_freq=test_df.l_freq_filter[row], 
                         h_freq=test_df.h_freq_filter[row], 
                         method='fir', phase='zero', verbose=0)

        #Create epoch object with our raw objects and events arrays
        channels_to_keep = [ch for ch in ch_names if 
                            ch not in test_df.channels_to_drop[row]]
        epoch_dict = {}
        for key, value in raw_dict.items():
            epoch_dict[key] = mne.Epochs(value, events=event_dict[key], 
                                        event_id=events_explained, 
                                        tmin=-3, tmax=test_df.tmax[row], 
                                        baseline=test_df.baseline_correction[row],
                                        preload=True,
                                        picks=channels_to_keep, verbose=0,
                                        detrend=test_df.detrend[row],
                                        reject=test_df.reject[row],
                                        flat=test_df.flat[row],
                                        reject_tmin=test_df.tmin[row],
                                        reject_tmax=test_df.tmax[row])

        #Skip creating projectors step to save compute time if not being
        #applied in this iteration
        if test_df.projectors_to_apply[row]:
            #Create dictionary of signal space projection vectors for each epoch
            proj_dict = {}
            for key, value in epoch_dict.items():
                proj_dict[key] = mne.compute_proj_epochs(value, 
                                                         n_eeg=2, 
                                                         verbose=0)
            #apply projectors
            for key, value in epoch_dict.items():
                value.add_proj(proj_dict[key][test_df.projectors_to_apply[row]], 
                               verbose=0)
                value.apply_proj(verbose=0)

        #Skip creating ICA components step to save compute time if not
        #being applied in this iteration
        if test_df.ica_to_exclude[row]:
            #create and fit ICA object to epochs
            for key, value in epoch_dict.items():
                ica = mne.preprocessing.ICA(n_components=5, method='picard', 
                                            max_iter='auto', verbose=0)
                ica.fit(value, verbose=0)
                #Apply the ICA
                ica.apply(value, exclude=test_df.ica_to_exclude[row],
                         verbose=0)

        #Resample the data at a new frequency, happens inplace
        for key, value in epoch_dict.items():
            value.resample(sfreq=test_df.selected_frequency[row])

        #Extract and standard scale data from all non-dropped epochs
        #Creates intermediate data dictionary
        int_data_dict = {}
        #Use robust sklearn scaler
        if test_df.scaler[row] == 'robust':
            mne_scaler = mne.decoding.Scaler(scalings='median')
            for key, value in epoch_dict.items():
                #with scalings=median implements sklearn robust scaler
                int_data_dict[key] = (mne_scaler.
                                      fit_transform(value.
                                                    get_data(tmin=test_df.tmin[row], 
                                                             tmax=test_df.tmax[row])))
        #No scaling option
        if test_df.scaler[row] is None:
            for key, value in epoch_dict.items():
                int_data_dict[key] = value.get_data(tmin=test_df.tmin[row], 
                                                      tmax=test_df.tmax[row])

        #Create updated dictionary of y values to reflect dropped epochs
        int_y_dict = {}
        for key, value in y_dict.items():
            #Process for individual row tests
            if ind_or_group == 'ind':
                if test_df.subject[row] in key:
                    temp_y_list = []
                    for i, epoch in enumerate(epoch_dict[key].drop_log):
                #MNE drop log shows empty parens for epochs that were not dropped - 
                #these are the trials we are keeping in each iteration
                        if epoch == ():
                            temp_y_list.append(value[i])
                    int_y_dict[key] = temp_y_list
            #Process for group tests
            if ind_or_group == 'group':
                if 'sesh_1' in key:
                    temp_y_list = []
                    for i, epoch in enumerate(epoch_dict[key].drop_log):
                #MNE drop log shows empty parens for epochs that were not dropped - 
                #these are the trials we are keeping in each iteration
                        if epoch == ():
                            temp_y_list.append(value[i])
                    int_y_dict[key] = temp_y_list

        #Assemble final y dict with only trials in our current combo
        #In each combo, coding 1st trial type to 0, 2nd trial type to 1
        final_y_dict = {}
        for key, value in int_y_dict.items():
            temp_y_list = []
            for y in value:
                if y == test_df.trial_combo[row][0]:
                    temp_y_list.append(0)
                if y == test_df.trial_combo[row][1]:
                    temp_y_list.append(1)
            final_y_dict[key] = np.array(temp_y_list)

        #Assemble data dict with only trials in our current combo
        final_data_dict = {}
        for key, value in int_data_dict.items():
            index_list = []
            for i, y in enumerate(int_y_dict[key]):
                if (y == test_df.trial_combo[row][0] or 
                    y == test_df.trial_combo[row][1]):
                    index_list.append(i)
            final_data_dict[key] = value[index_list]

        #Create csp_dict of csp objects
        csp_dict = {}
        for key, value in epoch_dict.items():
            csp_dict[key] = mne.decoding.CSP(n_components=int(test_df.n_components[row]), 
                                                 cov_est=test_df.cov_est[row], 
                                                 log=bool(test_df.log[row]));

        #Suppress output from this noisy function with no verbose option
        with io.capture_output() as captured:
        #Fit csp objects to training data from session 1        
            for key, value in csp_dict.items():
                #Try except to deal with iterations where fails to converge
                try:
                    value.fit(X=final_data_dict[key], 
                          y=final_y_dict[key]);
                except:
                    csp_dict[key] = 'CSP failed to converge'

        #Use csp objects to transform and save resulting data
        csp_data_dict = {}
        for key, value in csp_dict.items():
            #If except to deal with iterations where CSP fails to converge
            if value == 'CSP failed to converge':
                csp_data_dict[key] = 'CSP failed to converge'
            else:
                csp_data_dict[key] = value.transform(final_data_dict[key])

        #Model against our data for each subject and save the resulting score
        for key, value in csp_data_dict.items():

            #Process for individual row tests
            if ind_or_group == 'ind':
                #Pass through CSP failure to ouput
                if csp_dict[key] == 'CSP failed to converge':
                    test_df.at[row, 'test_acc'] = 'CSP failed to converge'
                #Else run cross val with LDA, return avg score
                else:
                    LDA = LinearDiscriminantAnalysis()
                    SKfold = StratifiedKFold(n_splits=5, 
                                             random_state=23, 
                                             shuffle=True)
                    cvs = cross_val_score(LDA, X=value, 
                                         y=final_y_dict[key], 
                                         cv=SKfold)
                    test_df.at[row, 'test_acc'] = np.mean(cvs)
                    
                    #Also save train_acc to avoid overfit
                    LDA.fit(value, final_y_dict[key])
                    test_df.at[row, 'train_acc'] = (LDA.score
                                                    (value, 
                                                     final_y_dict[key]))

            #Process for group tests
            if ind_or_group == 'group':
                #Pass through CSP failure to ouput
                if csp_dict[key] == 'CSP failed to converge':
                    subject = key[:5]
                    test_df.at[row, subject] = 'CSP failed to converge'
                #Else run cross val with LDA, return avg score
                else:
                    LDA = LinearDiscriminantAnalysis()
                    SKfold = StratifiedKFold(n_splits=5, 
                                             random_state=23, 
                                             shuffle=True)
                    cvs = cross_val_score(LDA, X=value, 
                                         y=final_y_dict[key], 
                                         cv=SKfold)
                    subject = key[:5]
                    test_df.at[row, subject] = np.mean(cvs)

        test_df.to_csv(f'data/{savefile}.csv', index=False)
        if row % 50 == 0:
            print(f'Grid search complete through row {row} of {test_df.shape[0]}')

In [10]:
#Test this first gridsearch
csp_grid_search(test_df=test_df, 
                ind_or_group='group', 
                savefile='csp_grid_search')

Grid search complete through row 0 of 96
Grid search complete through row 50 of 96


**Let's look at the results**

In [11]:
#Temporarily change pandas settings to see all columns
with pd.option_context("display.max_columns", None):
    #Change this code to match the subject of interest
    display(test_df[test_df['sub_J'].apply(type) != str].
            sort_values('sub_J', ascending=False)[:10])

,l_freq_filter,h_freq_filter,channels_to_drop,baseline_correction,projectors_to_apply,selected_frequency,tmin,tmax,detrend,reject,flat,ica_to_exclude,scaler,n_components,cov_est,log,trial_combo,sub_A,sub_C,sub_D,sub_E,sub_F,sub_G,sub_H,sub_J,sub_L
16,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,robust,6,concat,True,"(1, 5)",0.455238,0.843333,0.8375,0.871429,0.8125,0.798333,0.65,0.918095,0.9125
32,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,robust,8,concat,True,"(1, 5)",0.712381,0.782857,0.8125,0.887619,0.8375,0.785,0.74359,0.906667,0.8875
40,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,robust,8,epoch,True,"(1, 5)",0.74,0.782857,0.8125,0.807619,0.8375,0.785,0.737179,0.906667,0.8875
12,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,robust,4,epoch,False,"(1, 5)",0.597143,0.658333,0.7375,0.742857,0.7625,0.7725,0.630769,0.90381,0.85
4,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,robust,4,concat,False,"(1, 5)",0.514286,0.658333,0.7375,0.742857,0.7625,0.7725,0.630769,0.90381,0.85
92,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,None,8,epoch,False,"(1, 5)",0.62381,0.42381,CSP failed to converge,0.649524,CSP failed to converge,0.785,0.521795,0.902857,CSP failed to converge
84,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,None,8,concat,False,"(1, 5)",CSP failed to converge,0.437143,CSP failed to converge,CSP failed to converge,0.875,CSP failed to converge,CSP failed to converge,0.890476,CSP failed to converge
68,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,None,6,concat,False,"(1, 5)",CSP failed to converge,0.504762,0.825,0.541905,CSP failed to converge,CSP failed to converge,0.582051,0.886667,0.849167
49,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,None,4,concat,True,"(1, 4)",CSP failed to converge,CSP failed to converge,0.5125,CSP failed to converge,CSP failed to converge,0.518333,0.687179,0.88,CSP failed to converge
24,None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,{'eeg': 150},{'eeg': 20},None,robust,6,epoch,True,"(1, 5)",0.440952,0.742857,0.8375,0.846667,0.8125,0.798333,0.65,0.88,0.9


### Identifying per subject ideal individual CSP model settings

We'll be using these models based on CSP covariance matrices as one of our level 1 models in our final ensemble model. We have a bit more testing to do on those level 1 models though (notably individualizing the parameters to drop epochs), so let's examine our results thus far and extract the parameters we want to lock in for each subject. There are a few patterns here in the data:

- Most subjects have a particular trial combo that seems to be most differentiable for them
    - Not the same combination for every subject
    - This is likely influenced by the nature and area of their central nervous system injury
    - This is the most important parameter to improve modeling for each participant
- Most subjects clearly have a number of CSP components that outperform the others, but that number varies
- The robust scaler is uniformly better than no scaling of the data
    - CSP often fails to fit without scaled data
    - As such not saving down as an individual parameter - will just scale in all models
- The cov_est setting does not seems to matter much, but will take the highest performing setting for each person
- Log transforming appears to generally have a small impact on the CSP fit as well

We'll save the remaining, variable highest performing parameters for each individual to be used going forward as the core of their individual L1 model. The parameters we are saving are:

1. Trial combo
3. Covariance estimate method
4. Whether log transformed
5. Number of csp components

In [12]:
params_to_save = ['n_components',
                  'cov_est', 
                  'log', 
                  'trial_combo']

#Create function to save the info we want for a given subject
def pull_parameters(sub):
    #identify the highest score achieved for this subject
    sub_max = max(test_df[test_df[sub].apply(type) != str][sub])
    #Get the index in the dataframe where that occurred
    #In case of tie, returns first row
    [max_index] = test_df[test_df[sub] == sub_max].head(1).index
    #Save the params we want plus the subject from that best row
    temp_df = pd.DataFrame(test_df.loc[max_index, params_to_save]).T
    temp_df['subject'] = sub
    #Duplicate the row 25 times to join with our 25 epoch dropping combos
    return pd.concat([temp_df]*25, ignore_index=True)

#Use function to create the beginning of next gridsearch
sub_A_csp_df = pull_parameters('sub_A')
sub_C_csp_df = pull_parameters('sub_C')
sub_D_csp_df = pull_parameters('sub_D')
sub_E_csp_df = pull_parameters('sub_E')
sub_F_csp_df = pull_parameters('sub_F')
sub_G_csp_df = pull_parameters('sub_G')
sub_H_csp_df = pull_parameters('sub_H')
sub_J_csp_df = pull_parameters('sub_J')
sub_L_csp_df = pull_parameters('sub_L')

### Identifying per-subject epoch rejection criteria

The shapes of each individual's EEG recordings look very different - some have far more dramatic amplitude changes than others. The algorithm we are using to drop epochs is based on setting minimum and maximum amplitude differences between peaks and valleys in the data, so we need to personalize those settings if we want to be able to drop bad epochs in each individual study participants data.

We will quantify the percentage of epochs that are dropped from each experiment session with a range of rejection criteria, and then experiment with how dropping increasing percentages of epochs impacts our accuracy.

In [14]:
#Create list of epoch drop filters
reject_options_1 = [None]
reject_options_2 = [None, 
                  {'eeg': 40}, {'eeg': 50}, {'eeg': 60}, 
                  {'eeg': 70}, {'eeg': 80}, {'eeg': 90}, 
                  {'eeg': 105}, {'eeg': 140}, {'eeg': 175},
                   {'eeg': 135}, {'eeg': 145}, {'eeg': 150},
                   {'eeg': 155}, {'eeg': 160}, {'eeg': 167},
                   {'eeg': 146}, {'eeg': 147}, 
                   {'eeg': 130}, {'eeg': 125}, {'eeg': 120},
                   {'eeg': 115}, {'eeg': 110},
                   {'eeg': 180}, {'eeg': 185}, {'eeg': 190},
                   {'eeg': 143}, {'eeg': 110}, {'eeg': 115},
                   {'eeg': 120}, {'eeg': 125}, {'eeg': 130},
                   {'eeg': 83}, {'eeg': 86}, {'eeg': 78}, 
                   {'eeg': 100}, {'eeg': 95}, 
                   {'eeg': 210}, {'eeg': 183}, {'eeg': 200},
                   {'eeg': 205}]
flat_options_1 = [None]
flat_options_2 = [None, 
                {'eeg': 3}, {'eeg': 6}, {'eeg': 9}, 
                {'eeg': 11}, {'eeg': 13}, {'eeg': 15}, 
                {'eeg': 17}, {'eeg': 19}, {'eeg': 21},
                 {'eeg': 25}, {'eeg': 30}, {'eeg': 35},
                 {'eeg': 36}, {'eeg': 31}, {'eeg': 33}, 
                 {'eeg': 34}, {'eeg': 30.5}, {'eeg': 31.5},
                 {'eeg': 16}, {'eeg': 18}, {'eeg': 20},
                 {'eeg': 17.3}, {'eeg': 17.6}, {'eeg': 18.5},
                 {'eeg': 18.75}, {'eeg': 19.5}, {'eeg': 20.5},
                 {'eeg': 22}, {'eeg': 23}, {'eeg': 24}, 
                 {'eeg': 19.75},
                 {'eeg': 24.5}, {'eeg': 26}, {'eeg': 27},
                 {'eeg': 28}, {'eeg': 29}, 
                 {'eeg': 25.5}, {'eeg': 29.5}, {'eeg': 26.5},
                 {'eeg': 37}, {'eeg': 38}, {'eeg': 39},
                 {'eeg': 40}, {'eeg': 41}, {'eeg': 42}]

#Create dataframe - tests flat & reject filters independently
rejection_settings_df = pd.concat((pd.DataFrame(itertools.product(reject_options_1,
                                                                 flat_options_2), 
                                               columns=['reject', 'flat']),
                                  pd.DataFrame(itertools.product(reject_options_2,
                                                                 flat_options_1), 
                                               columns=['reject', 'flat'])),
                                 ignore_index=True)

#reindex
rejection_settings_df = rejection_settings_df.reindex(columns=['reject', 'flat'] + 
                                                      list(y_dict.keys()))

#Load raw dict with mne raw objects
raw_dict = {}
for key, value in data_dict.items():
    raw_dict[key] = mne.io.RawArray(value.T, info, verbose=0)

#Filter raw dict
for key, value in raw_dict.items():
    value.filter(l_freq=None, 
                 h_freq=40, 
                 method='fir', phase='zero', verbose=0)
            
#Compute percentage of trials dropped for each setting
for row in range(rejection_settings_df.shape[0]): 
    epoch_dict = {}
    for key, value in raw_dict.items():
        epoch_dict[key] = mne.Epochs(value, events=event_dict[key], 
                                     event_id=events_explained, 
                                     tmin=-3, tmax=4.5, 
                                     baseline=None,
                                     preload=True,
                                     picks=[ch for ch in ch_names if 
                                            ch not in ['AFz', 'F7', 'F8']],
                                     reject=rejection_settings_df.reject[row],
                                     flat=rejection_settings_df.flat[row],
                                     reject_tmin=1,
                                     reject_tmax=4.5,
                                     verbose=0)
    perc_trials_dropped_dict = {}
    for key, value in y_dict.items():
        dropped = value.shape[0] - epoch_dict[key].get_data().shape[0]
        drop_percentage = dropped / value.shape[0]
        rejection_settings_df.at[row, key] = drop_percentage

In [15]:
#view results, change subject to dig into data
subject = 'sub_C'
#Set exclude to ignore one of two filter types: 'reject' or 'flat'
exclude = 'reject'

(rejection_settings_df[['reject', 'flat', 
                        f'{subject}_sesh_1']].
 loc[(rejection_settings_df[f'{subject}_sesh_1'] < 0.2) & 
     (rejection_settings_df[f'{subject}_sesh_1'] > 0) &
 (rejection_settings_df[exclude].isna())].sort_values([f'{subject}_sesh_1']))

,reject,flat,sub_C_sesh_1
19,None,{'eeg': 16},0.005
7,None,{'eeg': 17},0.010
20,None,{'eeg': 18},0.010
22,None,{'eeg': 17.3},0.010
23,None,{'eeg': 17.6},0.010
24,None,{'eeg': 18.5},0.030
8,None,{'eeg': 19},0.035
25,None,{'eeg': 18.75},0.035
26,None,{'eeg': 19.5},0.060
21,None,{'eeg': 20},0.065


### Testing custom drop settings per subject with CSP model

For each subject, we will set out to test how our highest performing CSP model for that individual performs with rejection settings designed to drop the following percentages of epochs. For each approximate drop percentage we are targeting, we will use both min and max peak to trough settings to drop epochs that are too flat and too spiky at those percentage (checked via grid search of combinations.

- None (keep all epochs)
- Drop 1-3%
- Drop 3-5%
- Drop 5-10%
- Drop 10-15%

I populated the table below the the settings that most closely match the middle of each of those target windows. There is likely to be relatively wide variability in session 2 (our test data) compared to session 1, so there may be some subjects who have far more or far fewer epochs dropped in session 2. To avoid this in a production BCI device, it needs to either be trained on enough data to be resilient to inter-day swings in EEG signal, or be frequently rebiased or retrained in short calibration sessions.

**Note** Something about the implementation of this epoch rejection system in MNE seems to generate different results with the same code on occasion. I am discussing the issue with others on github, but for now I have to move forward as is. In an actual production environment, relying on this flat and drop system within MNE would not be workable.

**Creating list of drop criteria we want to test for each individual**
Based on the results of our grid search, let's manually assemble the list of settings we want to test for each individual that drop roughly the percentage of trials we are planning to drop.

In [16]:
#I want this to be a dictionary with keys equal to sub_A_flat, sub_A_reject, and then the list of values

sub_A_flat_options = [None, {'eeg': 37}, {'eeg': 40}, {'eeg': 41}, {'eeg': 42}]
sub_A_reject_options = [None, {'eeg': 207}, {'eeg': 200}, {'eeg': 190}, {'eeg': 183}]
sub_C_flat_options = [None, {'eeg': 19.5}, {'eeg': 20.5}, {'eeg': 22}, {'eeg': 23.5}]
sub_C_reject_options = [None, {'eeg': 205}, {'eeg': 200}, {'eeg': 190}, {'eeg': 175}]
sub_D_flat_options = [None, {'eeg': 24}, {'eeg': 25}, {'eeg': 26}, {'eeg': 27}]
sub_D_reject_options = [None, {'eeg': 200}, {'eeg': 190}, {'eeg': 175}, {'eeg': 155}]
sub_E_flat_options = [None, {'eeg': 22}, {'eeg': 23.5}, {'eeg': 25}, {'eeg': 26.5}]
sub_E_reject_options = [None, {'eeg': 190}, {'eeg': 185}, {'eeg': 175}, {'eeg': 150}]
sub_F_flat_options = [None, {'eeg': 26.5}, {'eeg': 29}, {'eeg': 30}, {'eeg': 31}]
sub_F_reject_options = [None, {'eeg': 120}, {'eeg': 110}, {'eeg': 100}, {'eeg': 95}]
sub_G_flat_options = [None, {'eeg': 31.5}, {'eeg': 33}, {'eeg': 34.5}, {'eeg': 35.3}]
sub_G_reject_options = [None, {'eeg': 205}, {'eeg': 146}, {'eeg': 130}, {'eeg': 110}]
sub_H_flat_options = [None, {'eeg': 21}, {'eeg': 22}, {'eeg': 23}, {'eeg': 24}]
sub_H_reject_options = [None, {'eeg': 220}, {'eeg': 210}, {'eeg': 200}, {'eeg': 190}]
sub_J_flat_options = [None, {'eeg': 21}, {'eeg': 21.5}, {'eeg': 22}, {'eeg': 23.5}]
sub_J_reject_options = [None, {'eeg': 185}, {'eeg': 175}, {'eeg': 155}, {'eeg': 140}]
sub_L_flat_options = [None, {'eeg': 29.5}, {'eeg': 31.5}, {'eeg': 32}, {'eeg': 32.5}]
sub_L_reject_options = [None, {'eeg': 175}, {'eeg': 115}, {'eeg': 100}, {'eeg': 90}]

**Assemble our test dataframe**
We'll combine our parameters identified in the trials above with the 25 combinations of flat and reject we want to test for each individual in the study, for a total of 25 tests per individual x 9 individuals = 225 iterations.

In [17]:
#Assemble subject A test df
sub_A_csp_df = (sub_A_csp_df.join(pd.DataFrame(itertools.product(sub_A_flat_options, 
                               sub_A_reject_options), 
                      columns=['flat', 'reject'])))

#Assemble subject C test df
sub_C_csp_df = sub_C_csp_df.join(pd.DataFrame(itertools.product(sub_C_flat_options, 
                               sub_C_reject_options), 
                      columns=['flat', 'reject']))

#Assemble subject D test df
sub_D_csp_df = sub_D_csp_df.join(pd.DataFrame(itertools.product(sub_D_flat_options, 
                               sub_D_reject_options), 
                      columns=['flat', 'reject']))

#Assemble subject E test df
sub_E_csp_df = sub_E_csp_df.join(pd.DataFrame(itertools.product(sub_E_flat_options, 
                               sub_E_reject_options), 
                      columns=['flat', 'reject']))

#Assemble subject F test df
sub_F_csp_df = sub_F_csp_df.join(pd.DataFrame(itertools.product(sub_F_flat_options, 
                               sub_F_reject_options), 
                      columns=['flat', 'reject']))

#Assemble subject G test df
sub_G_csp_df = sub_G_csp_df.join(pd.DataFrame(itertools.product(sub_G_flat_options, 
                               sub_G_reject_options), 
                      columns=['flat', 'reject']))

#Assemble subject H test df
sub_H_csp_df = sub_H_csp_df.join(pd.DataFrame(itertools.product(sub_H_flat_options, 
                               sub_H_reject_options), 
                      columns=['flat', 'reject']))

#Assemble subject J test df
sub_J_csp_df = sub_J_csp_df.join(pd.DataFrame(itertools.product(sub_J_flat_options, 
                               sub_J_reject_options), 
                      columns=['flat', 'reject']))

#Assemble subject L test df
sub_L_csp_df = sub_L_csp_df.join(pd.DataFrame(itertools.product(sub_L_flat_options, 
                               sub_L_reject_options), 
                      columns=['flat', 'reject']))

#Combine all test frames together
test_df_2 = pd.concat((sub_A_csp_df, 
                     sub_C_csp_df, 
                     sub_D_csp_df, 
                     sub_E_csp_df, 
                     sub_F_csp_df, 
                     sub_G_csp_df, 
                     sub_H_csp_df, 
                     sub_J_csp_df, 
                     sub_L_csp_df), 
                    ignore_index=True)

**Add the other parameters and results column to our test dataframe**
The rest of the parameters are identical for all subjects, so we can quickly add now at the end, along with a column to save our results.

In [19]:
#The settings we want to be the same for all trials
test_df_2['l_freq_filter'] = None
test_df_2['h_freq_filter'] = 40
test_df_2['channels_to_drop'] = [['AFz', 'F7', 'F8']]*test_df_2.shape[0]
test_df_2['baseline_correction'] = None
test_df_2['projectors_to_apply'] = slice(1)
test_df_2['selected_frequency'] = 256
test_df_2['tmin'] = 1
test_df_2['tmax'] = 4.5
test_df_2['detrend'] = None
test_df_2['ica_to_exclude'] = None
test_df_2['scaler'] = 'robust'

#Our results column
test_df_2['test_acc'] = None

In [23]:
test_df_2.shape

(225, 19)

### Now let's run our new test!

In [24]:
#Test this first gridsearch
csp_grid_search(test_df=test_df_2, 
                ind_or_group='ind', 
                savefile='csp_drop_combos_grid_search')

Grid search complete through row 0 of 225
Grid search complete through row 50 of 225
Grid search complete through row 100 of 225
Grid search complete through row 150 of 225
Grid search complete through row 200 of 225


In [25]:
#Review the results
test_df_2[test_df_2['subject'] == 'sub_L'].sort_values('test_acc', ascending=False)[:5]

,n_components,cov_est,log,trial_combo,subject,flat,reject,l_freq_filter,h_freq_filter,channels_to_drop,baseline_correction,projectors_to_apply,selected_frequency,tmin,tmax,detrend,ica_to_exclude,scaler,test_acc
208,8,concat,True,"(1, 4)",sub_L,{'eeg': 29.5},{'eeg': 100},None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,1.0
207,8,concat,True,"(1, 4)",sub_L,{'eeg': 29.5},{'eeg': 115},None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,1.0
219,8,concat,True,"(1, 4)",sub_L,{'eeg': 32},{'eeg': 90},None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,1.0
214,8,concat,True,"(1, 4)",sub_L,{'eeg': 31.5},{'eeg': 90},None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,1.0
209,8,concat,True,"(1, 4)",sub_L,{'eeg': 29.5},{'eeg': 90},None,40,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,1.0


### At this point, some of our results are looking very good!

Some of our LDA models are achieving 100 accuracy on test data from session 1. No doubt we'll lose quite a bit of accuracy in the shift to day 2 in our final assessment, but its very encouraging to see models performing well on test data.

We're going to once again pull our top parameters out for each subject, and then do one final set of tests. In our parameter searching notebook, the large grid search looking at many more preprocessing steps finally finished, and it looks like a few alternative settings might produce better results, and are worth trying here in our CSP models.

In [26]:
#Create function to save the best csp & drop params 
#for a given subject
def pull_parameters_2(sub):
    #get highest scoring row
    temp_df = (test_df_2[test_df_2['subject'] == sub]
               [['subject', 'n_components', 
                 'cov_est', 'log', 'trial_combo', 
                 'flat', 'reject', 'test_acc']].
               sort_values('test_acc', ascending=False)[:1])
    temp_df.drop('test_acc', axis=1, inplace=True)
    #Duplicate the row 128 times to join with our 128 other parameter combos
    return pd.concat([temp_df]*128, ignore_index=True)

#Create dataframes with the function
sub_A_csp_df_2 = pull_parameters_2('sub_A')
sub_C_csp_df_2 = pull_parameters_2('sub_C')
sub_D_csp_df_2 = pull_parameters_2('sub_D')
sub_E_csp_df_2 = pull_parameters_2('sub_E')
sub_F_csp_df_2 = pull_parameters_2('sub_F')
sub_G_csp_df_2 = pull_parameters_2('sub_G')
sub_H_csp_df_2 = pull_parameters_2('sub_H')
sub_J_csp_df_2 = pull_parameters_2('sub_J')
sub_L_csp_df_2 = pull_parameters_2('sub_L')

#The combinations of settings we want to try for all models
l_freq_filter_options = [None, 1.0]
h_freq_filter_options = [None, 40]
channels_to_drop_options = [['AFz', 'F7', 'F8'], 
                            ['AFz', 'F7', 'F8', 'T3', 
                             'T4', 'P7', 'PO3', 'O1', 
                             'PO4', 'P8', 'C4', 'P6']]
baseline_correction_options = [None]
projectors_to_apply_options = [slice(1), slice(1, 2, None)]
selected_frequency_options = [128, 256]
tmin_options = [0, 1]
tmax_options = [3, 4.5]
detrend_options = [None]
ica_to_exclude_options = [None]
scaler_options = ['robust']

#Names of corresponding columns in test df
columns = ['l_freq_filter',
           'h_freq_filter',
           'channels_to_drop',
           'baseline_correction',
           'projectors_to_apply',
           'selected_frequency',
           'tmin',
           'tmax',
           'detrend',
           'ica_to_exclude',
           'scaler']

#Create function to join our per-model params to general test set
def csp_param_joiner(dataframe):
    iter_list = itertools.product(l_freq_filter_options, 
                                  h_freq_filter_options, 
                                  channels_to_drop_options, 
                                  baseline_correction_options, 
                                  projectors_to_apply_options, 
                                  selected_frequency_options,
                                  tmin_options,
                                  tmax_options, 
                                  detrend_options,
                                  ica_to_exclude_options,
                                  scaler_options)
    return dataframe.join(pd.DataFrame(iter_list,
                                      columns=columns))

#Assemble our per-subject gridsearches
sub_A_csp_df_2 = csp_param_joiner(sub_A_csp_df_2)
sub_C_csp_df_2 = csp_param_joiner(sub_C_csp_df_2)
sub_D_csp_df_2 = csp_param_joiner(sub_D_csp_df_2)
sub_E_csp_df_2 = csp_param_joiner(sub_E_csp_df_2)
sub_F_csp_df_2 = csp_param_joiner(sub_F_csp_df_2)
sub_G_csp_df_2 = csp_param_joiner(sub_G_csp_df_2)
sub_H_csp_df_2 = csp_param_joiner(sub_H_csp_df_2)
sub_J_csp_df_2 = csp_param_joiner(sub_J_csp_df_2)
sub_L_csp_df_2 = csp_param_joiner(sub_L_csp_df_2)

#Combine all test frames together
test_df_3 = pd.concat((sub_A_csp_df_2, 
                       sub_C_csp_df_2, 
                       sub_D_csp_df_2, 
                       sub_E_csp_df_2, 
                       sub_F_csp_df_2, 
                       sub_G_csp_df_2, 
                       sub_H_csp_df_2, 
                       sub_J_csp_df_2, 
                       sub_L_csp_df_2), 
                      ignore_index=True)

#Add our empty column for accuracy to be filled in
test_df_3['test_acc'] = None

#Convert all NaNs to Nones so MNE can read them
test_df_3.replace(np.nan, None, inplace=True)

### Run the final iteration of tests to finalize our individual CSP models

In [27]:
csp_grid_search(test_df=test_df_3, 
                ind_or_group='ind', 
                savefile='final_csp_gridsearch')

Grid search complete through row 0 of 1152
Grid search complete through row 50 of 1152
Grid search complete through row 100 of 1152
Grid search complete through row 150 of 1152
Grid search complete through row 200 of 1152
Grid search complete through row 250 of 1152
Grid search complete through row 300 of 1152
Grid search complete through row 350 of 1152
Grid search complete through row 400 of 1152
Grid search complete through row 450 of 1152
Grid search complete through row 500 of 1152
Grid search complete through row 550 of 1152
Grid search complete through row 600 of 1152
Grid search complete through row 650 of 1152
Grid search complete through row 700 of 1152
Grid search complete through row 750 of 1152
Grid search complete through row 800 of 1152
Grid search complete through row 850 of 1152
Grid search complete through row 900 of 1152
Grid search complete through row 950 of 1152
Grid search complete through row 1000 of 1152
Grid search complete through row 1050 of 1152
Grid search

### Let's review our results

Our highest performers here will be our final CSP models and serve as one of our L1 models in our final ensemble model.

In [28]:
test_df_3[test_df_3['subject'] == 'sub_L'].sort_values('test_acc', ascending=False)[:5]

,subject,n_components,cov_est,log,trial_combo,flat,reject,l_freq_filter,h_freq_filter,channels_to_drop,baseline_correction,projectors_to_apply,selected_frequency,tmin,tmax,detrend,ica_to_exclude,scaler,test_acc
1066,sub_L,8,concat,True,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,40.0,"[AFz, F7, F8]",None,"slice(1, 2, None)",128,1,3.0,None,None,robust,1.0
1063,sub_L,8,concat,True,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,40.0,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,1.0
1039,sub_L,8,concat,True,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,"[AFz, F7, F8]",None,"slice(1, 2, None)",256,1,4.5,None,None,robust,1.0
1059,sub_L,8,concat,True,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,40.0,"[AFz, F7, F8]",None,"slice(None, 1, None)",128,1,4.5,None,None,robust,1.0
1127,sub_L,8,concat,True,"(1, 4)",{'eeg': 29.5},{'eeg': 100},1.0,40.0,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,1.0


### Let's save down these final model parameters for use in our ensemble model

In [29]:
#Save best params for sub A to dataframe to build from
final_csp_model_params = test_df_3[test_df_3['subject'] == 'sub_A'].sort_values('test_acc', ascending=False)[:1]

#Get list of subjects (except sub_A):
subj_list = [sub for sub in test_df_3.subject.unique() if sub != 'sub_A']

#Add best params for each 
for sub in subj_list:
    final_csp_model_params = pd.concat((final_csp_model_params, 
               test_df_3[test_df_3['subject'] == sub].sort_values('test_acc', ascending=False)[:1]),
              ignore_index=True)

final_csp_model_params

,subject,n_components,cov_est,log,trial_combo,flat,reject,l_freq_filter,h_freq_filter,channels_to_drop,baseline_correction,projectors_to_apply,selected_frequency,tmin,tmax,detrend,ica_to_exclude,scaler,test_acc
0,sub_A,8,epoch,True,"(2, 4)",{'eeg': 40},{'eeg': 183},1.0,40.0,"[AFz, F7, F8]",None,"slice(None, 1, None)",256,1,4.5,None,None,robust,0.90381
1,sub_C,8,concat,False,"(1, 3)",{'eeg': 22},{'eeg': 175},None,40.0,"[AFz, F7, F8]",None,"slice(None, 1, None)",128,1,4.5,None,None,robust,0.76
2,sub_D,6,concat,True,"(1, 3)",None,{'eeg': 200},None,40.0,"[AFz, F7, F8]",None,"slice(None, 1, None)",128,0,4.5,None,None,robust,0.893333
3,sub_E,8,concat,True,"(1, 5)",{'eeg': 25},{'eeg': 150},None,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...",None,"slice(1, 2, None)",256,1,3.0,None,None,robust,0.981818
4,sub_F,8,concat,True,"(2, 4)",{'eeg': 29},{'eeg': 100},1.0,40.0,"[AFz, F7, F8]",None,"slice(1, 2, None)",256,0,4.5,None,None,robust,0.985714
5,sub_G,6,concat,True,"(1, 5)",{'eeg': 35.3},{'eeg': 110},1.0,40.0,"[AFz, F7, F8]",None,"slice(None, 1, None)",128,1,4.5,None,None,robust,0.933333
6,sub_H,6,concat,False,"(2, 4)",{'eeg': 21},{'eeg': 190},None,None,"[AFz, F7, F8]",None,"slice(None, 1, None)",128,1,4.5,None,None,robust,0.841026
7,sub_J,6,concat,True,"(1, 5)",{'eeg': 22},{'eeg': 140},None,None,"[AFz, F7, F8]",None,"slice(1, 2, None)",128,1,3.0,None,None,robust,0.943636
8,sub_L,8,concat,True,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,40.0,"[AFz, F7, F8]",None,"slice(1, 2, None)",128,1,3.0,None,None,robust,1.0


In [30]:
#Save final params to csv for use in other notebooks
final_csp_model_params.to_csv('data/final_csp_model_params.csv', 
                              index=False)

### However, some changes need to be made before final model construction

As I was beginning to think about the final ensemble model, I realized that we are likely going to run into an issue if we use these parameters as constructed. While our LDA models are highly accurate, I know that our session 2 data is going to be somewhat varied from what we are training and testing on here in session 1 - EEG readings from the same person naturally vary from day to day (and even within the same day).

As such, if we use these final LDA models we have constructed, many of them will be 100% accurate on our testing data, our ensemble model will tilt entirely toward our LDA model, and we are likely to make some predictive errors on day 2 where the data is varied. If we had a larger dataset to work with, we would want extremely accurate models. However, that isn't the case here, and our training dataset is very small, so we need to make a broader array of less specific models to feed into our ensemble.

As a result, I will be limiting the number of CSP components to make these CSP/LDA models less highly fit and more generalizable.

Additionally, we are going to use identical MNE parameters for all the L1 models used for each subject to speed our ensemble model. As a result, I am going to move tmin to 1 in all cases (to help our LDA models and avoid the change of waves from stimulus initiation), but make tmax at least 4.5 to give our CNN models enough to look at.

Finally, I will also be mandating that the longer list of channels be dropped for every subject. The original study authors examined all the epochs to manually drop channels, even in session 2, so I am going to err on the side of caution and make sure no surprise bad channels in session 2 throw off our models. Additionally, lowering the dimensionality of the input data should also reduce overfitting.

So, let's set up a new grid search with this new set of parameters.

In [3]:
locked_params = pd.read_csv('data/final_csp_model_params.csv')
variable_columns = ['n_components', 'cov_est', 'log', 'l_freq_filter',
                   'h_freq_filter', 'channels_to_drop', 'projectors_to_apply',
                   'selected_frequency', 'tmin', 'tmax']
#Drop columns we will vary
locked_params.drop(variable_columns, axis=1, inplace=True)


#Reset trial combo to tuple, flat and reject to dicts
locked_params['trial_combo'] = (locked_params['trial_combo'].
                                apply(lambda x: literal_eval(x)))
#Use list comprehension b/c NaN can appear in list
locked_params['flat'] = [None if pd.isna(x) else literal_eval(x)
                         for x in locked_params['flat']]
#Use list comprehension b/c NaN can appear in list
locked_params['reject'] = [None if pd.isna(x) else literal_eval(x)
                         for x in locked_params['reject']]

In [12]:
n_components_options = [2, 3]
cov_est_options = ['concat']
log_options = [True, False]
l_freq_filter_options = [1.0]
h_freq_filter_options = [None, 40]
channels_to_drop_options = [['AFz', 'F7', 'F8', 'T3', 
                             'T4', 'P7', 'PO3', 'O1', 
                             'PO4', 'P8', 'C4', 'P6']]
projectors_to_apply_options = [slice(1), slice(1, 2, None)]
selected_frequency_options = [256]
tmin_options = [1]
tmax_options = [4.5, 5.5]

#Get number of permutations of variable parameters
permutations = len(list(itertools.
                        product(n_components_options,
                                cov_est_options,
                                log_options,
                                l_freq_filter_options,
                                h_freq_filter_options,
                                channels_to_drop_options,
                                projectors_to_apply_options,
                                selected_frequency_options,
                                tmin_options,
                                tmax_options)))

#Create dataframe of our set of variable params
variable_params = pd.DataFrame(itertools.
                               product(n_components_options,
                                cov_est_options,
                                log_options,
                                l_freq_filter_options,
                                h_freq_filter_options,
                                channels_to_drop_options,
                                projectors_to_apply_options,
                                selected_frequency_options,
                                tmin_options,
                                tmax_options), 
                           columns=variable_columns)

#Duplicate each row of our locked params to match the number of variable
#permutations we are going to run
locked_temp = pd.DataFrame(np.repeat(locked_params.values, 
                                     permutations, 
                                     axis=0))
locked_temp.columns = locked_params.columns

#Concat variable params lists with itself (subjects - 1) times to get
#DF with them repeated for every subject
for i in range(locked_params.shape[0] - 1):
    variable_params = pd.concat((variable_params, variable_params),
                              ignore_index=True)

#Join our locked and variable parameters
test_df_4 = locked_temp.join(variable_params)

#Convert all NaNs to Nones so MNE can read them
test_df_4.replace(np.nan, None, inplace=True)

test_df_4.shape

(288, 19)

In [16]:
csp_grid_search(test_df=test_df_4, 
                ind_or_group='ind', 
                savefile='csp_gridsearch_low_components')

Grid search complete through row 0 of 288
Grid search complete through row 50 of 288
Grid search complete through row 100 of 288
Grid search complete through row 150 of 288
Grid search complete through row 200 of 288
Grid search complete through row 250 of 288


**Let's view our results**

In [45]:
test_df_4[test_df_4['subject'] == 'sub_L'].sort_values('train_acc', ascending=True)[:10]

,subject,trial_combo,flat,reject,baseline_correction,detrend,ica_to_exclude,scaler,test_acc,n_components,cov_est,log,l_freq_filter,h_freq_filter,channels_to_drop,projectors_to_apply,selected_frequency,tmin,tmax,train_acc
271,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.942857,2,concat,False,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.942857
283,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.971429,3,concat,False,1.0,None,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.942857
269,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.942857,2,concat,False,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.942857
267,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.942857,2,concat,False,1.0,None,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.942857
265,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.942857,2,concat,False,1.0,None,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.942857
287,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.971429,3,concat,False,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.942857
264,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.952564,2,concat,False,1.0,None,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,4.5,0.952381
268,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.952564,2,concat,False,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,4.5,0.952381
279,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.942857,3,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.957143
263,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.957143,2,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.957143


**Based on the dataframe above, let's assemble our selections of final CSP models**

Looked through the list of output manually to find models that were very similar on train and test accuracy and 75-80% accurate.

In [46]:
selected_index_list = [5, 52, 85, 99, 133, 173, 213, 229, 271]

**Sub D's results were much worse than the rest - I'm going to manually bump it up to 4 CSP components in the resulting df**

**Sub L had the reverse problem - all the results were way too good. Going to bump it down to 1 CSP component**

In [52]:
test_df_4.at[85, 'n_components'] = 4
test_df_4.at[271, 'n_components'] = 1
test_df_4.iloc[selected_index_list]

,subject,trial_combo,flat,reject,baseline_correction,detrend,ica_to_exclude,scaler,test_acc,n_components,cov_est,log,l_freq_filter,h_freq_filter,channels_to_drop,projectors_to_apply,selected_frequency,tmin,tmax,train_acc
5,sub_A,"(2, 4)",{'eeg': 40},{'eeg': 183},None,None,None,robust,0.810833,2,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.807692
52,sub_C,"(1, 3)",{'eeg': 22},{'eeg': 175},None,None,None,robust,0.6575,3,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,4.5,0.645570
85,sub_D,"(1, 3)",None,{'eeg': 200},None,None,None,robust,0.525,4,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.587500
99,sub_E,"(1, 5)",{'eeg': 25},{'eeg': 150},None,None,None,robust,0.723077,2,concat,True,1.0,None,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.753846
133,sub_F,"(2, 4)",{'eeg': 29},{'eeg': 100},None,None,None,robust,0.771429,2,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.771429
173,sub_G,"(1, 5)",{'eeg': 35.3},{'eeg': 110},None,None,None,robust,0.762637,2,concat,False,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.764706
213,sub_H,"(2, 4)",{'eeg': 21},{'eeg': 190},None,None,None,robust,0.7,3,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.728571
229,sub_J,"(1, 5)",{'eeg': 22},{'eeg': 140},None,None,None,robust,0.773333,2,concat,True,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(None, 1, None)",256,1,5.5,0.786667
271,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.942857,1,concat,False,1.0,40.0,"[AFz, F7, F8, T3, T4, P7, PO3, O1, PO4, P8, C4...","slice(1, 2, None)",256,1,5.5,0.942857


In [53]:
test_df_4.iloc[selected_index_list].to_csv('data/csp_models_not_overfit.csv',
                                          index=False)